In [ ]:
import openai
import psycopg2
import pandas as pd
from sentence_transformers import SentenceTransformer
import os, re
import requests, uuid, json
import numpy as np

In [ ]:
openai.api_type = "azure"
openai.api_base = "https://fusion-gpt-training.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "a83037e20d3d4d7982b0c52080e730fb"

In [ ]:
host = "pureelawvec.postgres.database.azure.com"
dbname = "postgres"
user = "pgadmin"
password = "Puree1234"
sslmode = "require"
# Construct connection string

In [ ]:
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
conn = psycopg2.connect(conn_string) 
print("Connection established")
cursor = conn.cursor()
# Fetch all rows from table

In [ ]:
df = pd.read_csv("law.csv")
df['Content'] = df['Content'].str.replace("\n", " ")
# Extract sections using regular expressions
pattern = r'Section \d+\..*?(?=Section \d+\.|$)'
df['Section'] = df['Content'].apply(lambda x: re.findall(pattern, x, re.DOTALL))
# Explode the list of sections into separate rows
df = df.explode('Section')
df_text = df.drop('Content', axis= 1)
# # Reset index
df_text.reset_index(drop=True, inplace=True)
df = df_text.dropna()

In [ ]:
x = np.squeeze(df.Section.values)
sentence_list = [x for x in x]
sentence_list

In [ ]:
translated = []
for item in sentence_list:
    translated.append(eng2thai(item))
translated

In [ ]:
from sentence_transformers import SentenceTransformer
embed_list = []
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentence_list)
embed_list.append(embeddings.tolist())
print(embeddings)

In [ ]:
try:
    # cursor.execute("CREATE EXTENSION IF NOT EXISTS pg_trgm;")
    cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    cursor.execute(""" 
    CREATE TABLE Criminal_Law(
        id INTEGER PRIMARY KEY,
        embedding vector,
        text_eng TEXT,
        text_thai TEXT
    );""")
    for i in range(len(sentence_list)):  
        cursor.execute("INSERT INTO Criminal_Law(id, embedding, text_eng, text_thai) VALUES (%s, %s, %s, %s);",   
                    (i, embed_list[0][i], sentence_list[i], translated[i]))
        print(sentence_list[i]) 
        print(embed_list[0][i])
        conn.commit()  
except Exception as e:
        print(f"Error: {e}")
        conn.rollback()

In [ ]:
def text2vec(input):
    embed_list = []
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    embeddings = model.encode(input)
    embed_list.append(embeddings.tolist())
    return embed_list[0]

def search_vectordb(question):# Query vector
    search_text = []
    try:
         cursor.execute(f"""SELECT text_thai, cosine_distance('{text2vec(question)}', embedding) as cosign
         FROM Criminal_Law
         ORDER BY cosine_distance('{text2vec(question)}', embedding) ASC LIMIT 7
         """)
         query = cursor.fetchall()
         text_eng_results = [row[0] for row in query] 
         return text_eng_results
    except Exception as e:  
            print(f"Error: {e}")
            conn.rollback()
    return "null"